# Chapter 6: K 近邻算法 (KNN) 实践

## 实验概述

在本实验中，你将使用 **Digits 数据集**（手写数字识别）来学习 K 近邻算法的实现和应用。Digits 数据集包含 1797 个 8x8 像素的手写数字图像，每个图像对应 0-9 中的一个数字。

### 实验目标

1. 理解 KNN 算法的基本原理
2. 实现欧氏距离计算
3. 手动实现 KNN 分类器
4. 使用 scikit-learn 的 KNN 模型
5. 找到最优的 K 值
6. 可视化决策边界和混淆矩阵

### 数据集信息

- **样本数量**: 1797
- **特征数量**: 64 (8x8 像素)
- **类别数量**: 10 (数字 0-9)
- **特征**: 每个特征的值是 0-16 之间的灰度值

## Part 1: 数据加载与探索

### 1.1 导入必要的库

In [ ]:
# ===== 预填充代码 =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

print("库导入成功！")

### 1.2 加载 Digits 数据集

In [ ]:
# ===== 预填充代码 =====
# 加载数据集
digits = load_digits()
X = digits.data
y = digits.target

print("数据集信息:")
print(f"样本数量: {X.shape[0]}")
print(f"特征数量: {X.shape[1]}")
print(f"图像尺寸: 8x8")
print(f"类别数量: {len(np.unique(y))}")
print(f"\n类别: {np.unique(y)}")
print(f"\n特征值范围: [{X.min():.1f}, {X.max():.1f}]")

### 1.3 可视化手写数字图像

In [ ]:
# ===== 预填充代码 =====
# 可视化前 20 个数字图像
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
axes = axes.ravel()

for i in range(20):
    axes[i].imshow(digits.images[i], cmap='gray')
    axes[i].set_title(f'标签: {y[i]}')
    axes[i].axis('off')

plt.suptitle('Digits 数据集 - 前20个样本', fontsize=14)
plt.tight_layout()
plt.show()

### 1.4 数据集统计

**任务**: 统计每个数字的样本数量

In [ ]:
# ===== 参考答案 =====
# 统计每个数字的样本数量
# 使用 np.bincount() 统计每个类别的样本数
label_counts = np.bincount(y)

print("每个数字的样本数量:")
for digit, count in enumerate(label_counts):
    print(f"  数字 {digit}: {count} 个样本")

# 可视化类别分布
plt.figure(figsize=(10, 5))
bars = plt.bar(range(10), label_counts, color='skyblue', edgecolor='black', alpha=0.7)

# 在柱子上方添加数值标签
for bar, count in zip(bars, label_counts):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
             f'{count}', ha='center', va='bottom', fontweight='bold')

plt.xlabel('数字')
plt.ylabel('样本数量')
plt.title('Digits 数据集 - 类别分布')
plt.xticks(range(10))
plt.grid(True, alpha=0.3, axis='y')

# 添加平均线
mean_count = label_counts.mean()
plt.axhline(y=mean_count, color='red', linestyle='--', 
            label=f'平均: {mean_count:.0f}')
plt.legend()

plt.tight_layout()
plt.show()

# 统计信息
print(f"\n统计信息:")
print(f"总样本数: {len(y)}")
print(f"最少类别样本数: {label_counts.min()}")
print(f"最多类别样本数: {label_counts.max()}")
print(f"标准差: {label_counts.std():.2f}")
print(f"数据集是否平衡: {'是' if label_counts.std() < 20 else '否'}")

### 1.5 数据预处理与划分

In [ ]:
# ===== 预填充代码 =====
# 特征归一化 (0-1 范围)
X_normalized = X / 16.0  # 像素值范围是 0-16

# 划分数据集 (80% 训练, 20% 测试)
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42, stratify=y
)

print(f"训练集大小: {X_train.shape}")
print(f"测试集大小: {X_test.shape}")
print(f"\n特征值范围 (归一化后): [{X_normalized.min():.1f}, {X_normalized.max():.1f}]")

## Part 2: 距离度量实现

### 2.1 欧氏距离

**任务**: 实现欧氏距离函数 $d(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$

In [ ]:
# ===== 参考答案 =====
def euclidean_distance(x1, x2):
    """
    计算两个向量之间的欧氏距离
    
    参数:
    x1: 第一个向量 (n_features,)
    x2: 第二个向量 (n_features,)
    
    返回:
    欧氏距离
    """
    # 计算欧氏距离：sqrt(sum((x1 - x2)^2))
    return np.sqrt(np.sum((x1 - x2) ** 2))

# 测试欧氏距离函数
a = np.array([1, 2, 3, 4])
b = np.array([5, 6, 7, 8])
dist = euclidean_distance(a, b)
print(f"欧氏距离测试: d({a}, {b}) = {dist:.4f}")
print(f"手动计算: sqrt((1-5)^2 + (2-6)^2 + (3-7)^2 + (4-8)^2) = sqrt(16+16+16+16) = sqrt(64) = 8.0")
# 预期输出: 8.0

### 2.2 曼哈顿距离

**任务**: 实现曼哈顿距离函数 $d(x, y) = \sum_{i=1}^{n}|x_i - y_i|$

In [ ]:
# ===== 参考答案 =====
def manhattan_distance(x1, x2):
    """
    计算两个向量之间的曼哈顿距离
    
    参数:
    x1: 第一个向量
    x2: 第二个向量
    
    返回:
    曼哈顿距离
    """
    # 计算曼哈顿距离：sum(|x1 - x2|)
    return np.sum(np.abs(x1 - x2))

# 测试曼哈顿距离函数
dist_m = manhattan_distance(a, b)
print(f"曼哈顿距离测试: d({a}, {b}) = {dist_m:.4f}")
print(f"手动计算: |1-5| + |2-6| + |3-7| + |4-8| = 4 + 4 + 4 + 4 = 16.0")
# 预期输出: 16.0

### 2.3 批量距离计算

**任务**: 计算一个样本到所有训练样本的距离

In [ ]:
# ===== 参考答案 =====
def compute_distances(X_train, x_test, metric='euclidean'):
    """
    计算测试样本到所有训练样本的距离
    
    参数:
    X_train: 训练集 (n_samples, n_features)
    x_test: 测试样本 (n_features,)
    metric: 距离度量 ('euclidean' 或 'manhattan')
    
    返回:
    距离数组 (n_samples,)
    """
    distances = []
    
    # 根据选择的距离度量函数
    if metric == 'euclidean':
        distance_func = euclidean_distance
    elif metric == 'manhattan':
        distance_func = manhattan_distance
    else:
        raise ValueError("不支持的距离度量")
    
    # 遍历训练集，计算每个样本到测试样本的距离
    for x_train in X_train:
        dist = distance_func(x_train, x_test)
        distances.append(dist)
    
    return np.array(distances)

# 测试
x_sample = X_test[0]
dists = compute_distances(X_train, x_sample, metric='euclidean')
print(f"测试样本到训练集的距离 (前10个): {dists[:10]}")
print(f"最小距离: {dists.min():.4f}")
print(f"最大距离: {dists.max():.4f}")
print(f"平均距离: {dists.mean():.4f}")

# 测试曼哈顿距离
dists_m = compute_distances(X_train, x_sample, metric='manhattan')
print(f"\n曼哈顿距离 - 前10个: {dists_m[:10]}")
print(f"曼哈顿距离 - 最小: {dists_m.min():.4f}")

## Part 3: 手动实现 KNN 分类器

### 3.1 实现预测函数

**任务**: 实现 KNN 预测函数

In [ ]:
# ===== 参考答案 =====
def knn_predict(X_train, y_train, x_test, k=5):
    """
    KNN 分类预测
    
    参数:
    X_train: 训练集特征
    y_train: 训练集标签
    x_test: 测试样本
    k: 邻居数量
    
    返回:
    预测类别
    """
    # 1. 计算测试样本到所有训练样本的距离
    distances = compute_distances(X_train, x_test, metric='euclidean')
    
    # 2. 获取距离最近的 k 个样本的索引
    k_indices = np.argsort(distances)[:k]
    
    # 3. 获取 k 个最近邻的标签
    k_nearest_labels = y_train[k_indices]
    
    # 4. 多数投票，返回最常见的类别
    label_counts = Counter(k_nearest_labels)
    prediction = label_counts.most_common(1)[0][0]
    
    return prediction

# 测试 KNN 预测
x_test_sample = X_test[0]
true_label = y_test[0]
pred_label = knn_predict(X_train, y_train, x_test_sample, k=5)

print(f"测试样本索引: 0")
print(f"真实标签: {true_label}")
print(f"预测标签: {pred_label}")
print(f"预测{'正确' if true_label == pred_label else '错误'}!")

# 显示5个最近邻的标签
distances = compute_distances(X_train, x_test_sample, metric='euclidean')
k_indices = np.argsort(distances)[:5]
k_nearest_labels = y_train[k_indices]

print(f"\n5个最近邻的标签: {k_nearest_labels}")
print(f"邻居距离: {distances[k_indices]}")
print(f"投票结果: {Counter(k_nearest_labels)}")

### 3.2 批量预测

In [ ]:
# ===== 参考答案 =====
def knn_predict_batch(X_train, y_train, X_test, k=5):
    """
    对测试集进行批量预测
    
    参数:
    X_train: 训练集特征
    y_train: 训练集标签
    X_test: 测试集特征
    k: 邻居数量
    
    返回:
    预测标签数组
    """
    predictions = []
    
    # 遍历测试集，对每个样本进行预测
    for i, x_test in enumerate(X_test):
        pred = knn_predict(X_train, y_train, x_test, k)
        predictions.append(pred)
        
        # 每100个样本打印一次进度
        if (i + 1) % 100 == 0:
            print(f"已处理 {i+1}/{len(X_test)} 个样本")
    
    return np.array(predictions)

# 在小测试集上测试（只取前20个样本加快速度）
print("在测试集子集上进行预测（前20个样本）...")
y_pred_manual = knn_predict_batch(X_train, y_train, X_test[:20], k=5)

accuracy = np.mean(y_pred_manual == y_test[:20])
print(f"\n手动实现的 KNN 准确率: {accuracy:.4f}")

# 显示详细的预测结果
print("\n详细预测结果:")
for i in range(len(y_test[:20])):
    print(f"样本 {i}: 真实={y_test[i]}, 预测={y_pred_manual[i]}, {'✓' if y_test[i] == y_pred_manual[i] else '✗'}")

## Part 4: 使用 scikit-learn 的 KNN

### 4.1 创建 KNN 分类器

In [ ]:
# ===== 参考答案 =====
from sklearn.neighbors import KNeighborsClassifier

# 创建 KNN 分类器
knn = KNeighborsClassifier(n_neighbors=5)

# 训练模型（注意：KNN 是惰性学习，训练过程只是存储数据）
knn.fit(X_train, y_train)

# 在训练集和测试集上预测
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# 计算准确率
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"KNN 分类器 (K=5):")
print(f"  训练集准确率: {train_acc:.4f}")
print(f"  测试集准确率: {test_acc:.4f}")
print(f"  过拟合情况: {'是' if train_acc - test_acc > 0.05 else '否'}")

# 显示部分预测结果
print("\n前10个测试样本的预测结果:")
for i in range(10):
    print(f"样本 {i}: 真实={y_test[i]}, 预测={y_test_pred[i]}, {'✓' if y_test[i] == y_test_pred[i] else '✗'}")

### 4.2 寻找最优 K 值

**任务**: 尝试不同的 K 值，找到最优的 K

In [ ]:
# ===== 参考答案 =====
# 尝试不同的 K 值
k_values = list(range(1, 31, 2))  # 1, 3, 5, ..., 29
train_scores = []
test_scores = []

# 遍历不同的 K 值，记录训练集和测试集的准确率
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    train_acc = knn.score(X_train, y_train)
    test_acc = knn.score(X_test, y_test)
    
    train_scores.append(train_acc)
    test_scores.append(test_acc)
    
    print(f"K={k:2d}: 训练准确率={train_acc:.4f}, 测试准确率={test_acc:.4f}")

# 找到最佳 K 值（测试集准确率最高）
best_k_idx = np.argmax(test_scores)
best_k = k_values[best_k_idx]
best_score = test_scores[best_k_idx]

print(f"\n最佳 K 值: {best_k}")
print(f"最佳测试集准确率: {best_score:.4f}")

# 绘制 K 值与准确率的关系
plt.figure(figsize=(12, 6))
plt.plot(k_values, train_scores, 'o-', label='训练集准确率', linewidth=2, markersize=6)
plt.plot(k_values, test_scores, 's-', label='测试集准确率', linewidth=2, markersize=6)
plt.axvline(x=best_k, color='green', linestyle='--', linewidth=2, label=f'最佳 K={best_k}')
plt.xlabel('K 值', fontsize=12)
plt.ylabel('准确率', fontsize=12)
plt.title('K 值对模型性能的影响', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(k_values)

# 在图中标注关键信息
plt.annotate(f'最高测试准确率\n{best_score:.3f}', 
             xy=(best_k, best_score),
             xytext=(best_k + 5, best_score - 0.02),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=10)

plt.tight_layout()
plt.show()

# 分析结果
print("\n分析结果:")
print(f"K=1 时的训练准确率: {train_scores[0]:.4f}")
print(f"K=29 时的训练准确率: {train_scores[-1]:.4f}")
print(f"K=1 时的测试准确率: {test_scores[0]:.4f}")
print(f"K=29 时的测试准确率: {test_scores[-1]:.4f}")
print(f"\n训练准确率下降: {train_scores[0] - train_scores[-1]:.4f}")
print(f"测试准确率变化: {test_scores[0] - test_scores[-1]:.4f}")

### 4.3 加权 KNN

**任务**: 比较均匀权重和距离权重

In [ ]:
# ===== 参考答案 =====
# 创建两个 KNN 模型：均匀权重和距离权重

# 均匀权重：所有邻居的权重相同
knn_uniform = KNeighborsClassifier(n_neighbors=best_k, weights='uniform')

# 距离权重：距离越近的邻居权重越大
# 权重 = 1 / distance
knn_distance = KNeighborsClassifier(n_neighbors=best_k, weights='distance')

# 训练并评估
knn_uniform.fit(X_train, y_train)
knn_distance.fit(X_train, y_train)

uniform_acc = knn_uniform.score(X_test, y_test)
distance_acc = knn_distance.score(X_test, y_test)

print("权重策略比较:")
print(f"  均匀权重 (uniform): {uniform_acc:.4f}")
print(f"  距离权重 (distance): {distance_acc:.4f}")
print(f"  差异: {distance_acc - uniform_acc:+.4f}")

# 分析
if distance_acc > uniform_acc:
    print("\n结论: 距离权重表现更好，说明近邻样本更重要")
elif distance_acc < uniform_acc:
    print("\n结论: 均匀权重表现更好，可能是噪声样本影响")
else:
    print("\n结论: 两种策略效果相当")

### 4.4 不同距离度量的比较

In [ ]:
# ===== 预填充代码 =====
# 比较不同的距离度量
metrics = ['euclidean', 'manhattan', 'minkowski']
results = []

for metric in metrics:
    knn = KNeighborsClassifier(n_neighbors=best_k, metric=metric)
    knn.fit(X_train, y_train)
    
    train_acc = knn.score(X_train, y_train)
    test_acc = knn.score(X_test, y_test)
    
    results.append({
        'metric': metric,
        'train_acc': train_acc,
        'test_acc': test_acc
    })

results_df = pd.DataFrame(results)
print("不同距离度量的比较:")
print(results_df)

## Part 5: 模型评估与可视化

### 5.1 使用最佳模型进行预测

In [ ]:
# ===== 参考答案 =====
# 使用最佳参数创建模型
best_knn = KNeighborsClassifier(n_neighbors=best_k)

# 训练模型
best_knn.fit(X_train, y_train)

# 进行预测
y_pred_best = best_knn.predict(X_test)

# 显示分类报告
print("分类报告:")
print(classification_report(y_test, y_pred_best, digits=4))

print(f"测试集准确率: {accuracy_score(y_test, y_pred_best):.4f}")

# 显示每个类别的准确率
for digit in range(10):
    mask = y_test == digit
    if mask.sum() > 0:
        acc_digit = accuracy_score(y_test[mask], y_pred_best[mask])
        print(f"  数字 {digit} 的准确率: {acc_digit:.4f}")

### 5.2 混淆矩阵可视化

In [ ]:
# ===== 参考答案 =====
# 计算混淆矩阵
cm = confusion_matrix(y_test, y_pred_best)

# 可视化混淆矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('预测类别')
plt.ylabel('真实类别')
plt.title('混淆矩阵 - 手写数字识别')
plt.show()

# 分析混淆矩阵
print("混淆矩阵分析:")
print(f"对角线元素和 (正确预测数): {np.trace(cm)}")
print(f"总样本数: {cm.sum()}")
print(f"准确率: {np.trace(cm) / cm.sum():.4f}")

# 找出最容易混淆的数字对
print("\n最易混淆的数字对 (非对角线元素):")
confusion_pairs = []
for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 0:
            confusion_pairs.append((i, j, cm[i, j]))

# 按混淆数量排序
confusion_pairs.sort(key=lambda x: x[2], reverse=True)
for real, pred, count in confusion_pairs[:5]:
    print(f"  数字 {real} 被预测为 {pred}: {count} 次")

### 5.3 可视化预测结果

**任务**: 显示一些预测正确和错误的样本

In [ ]:
# ===== 参考答案 =====
# 找出预测正确和错误的样本
correct_mask = y_pred_best == y_test
correct_indices = np.where(correct_mask)[0]
incorrect_indices = np.where(~correct_mask)[0]

print(f"预测正确数量: {len(correct_indices)}")
print(f"预测错误数量: {len(incorrect_indices)}")
print(f"准确率: {len(correct_indices) / (len(correct_indices) + len(incorrect_indices)):.4f}")

# 可视化一些预测结果
fig, axes = plt.subplots(2, 5, figsize=(14, 6))

# 第一行: 预测正确的样本
for i in range(5):
    idx = correct_indices[i]
    img = X_test[idx].reshape(8, 8)
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'真实:{y_test[idx]}, 预测:{y_pred_best[idx]}')
    axes[0, i].axis('off')

# 第二行: 预测错误的样本
for i in range(5):
    if i < len(incorrect_indices):
        idx = incorrect_indices[i]
        img = X_test[idx].reshape(8, 8)
        axes[1, i].imshow(img, cmap='gray')
        axes[1, i].set_title(f'真:{y_test[idx]}, 预:{y_pred_best[idx]}', color='red')
        axes[1, i].axis('off')

axes[0, 0].set_ylabel('预测正确', fontsize=12)
axes[1, 0].set_ylabel('预测错误', fontsize=12)
plt.suptitle('KNN 预测结果示例', fontsize=14)
plt.tight_layout()
plt.show()

# 分析错误预测
print("\n错误预测分析:")
for i in range(min(5, len(incorrect_indices))):
    idx = incorrect_indices[i]
    print(f"样本 {idx}: 真实={y_test[idx]}, 预测={y_pred_best[idx]}")

## Part 6: 挑战练习

### 6.1 交叉验证

**任务**: 使用交叉验证评估模型稳定性

In [ ]:
# ===== 参考答案 =====
# 使用 5 折交叉验证评估模型
cv_scores = cross_val_score(
    KNeighborsClassifier(n_neighbors=best_k),
    X_normalized, y,
    cv=5,
    scoring='accuracy'
)

print("5 折交叉验证结果:")
print(f"每折分数: {cv_scores}")
print(f"平均分数: {cv_scores.mean():.4f}")
print(f"标准差: {cv_scores.std():.4f}")
print(f"最小值: {cv_scores.min():.4f}")
print(f"最大值: {cv_scores.max():.4f}")
print(f"变异系数: {cv_scores.std() / cv_scores.mean() * 100:.2f}%")

# 可视化
plt.figure(figsize=(10, 5))
bars = plt.bar(range(1, 6), cv_scores, alpha=0.7, edgecolor='black', 
               color='steelblue', linewidth=2)
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--', linewidth=2,
            label=f'平均分: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}')
plt.xlabel('折数', fontsize=12)
plt.ylabel('准确率', fontsize=12)
plt.title('5 折交叉验证结果', fontsize=14)
plt.xticks(range(1, 6))
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')
plt.ylim([cv_scores.min() - 0.02, cv_scores.max() + 0.02])

# 在柱子上添加数值
for bar, score in zip(bars, cv_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
             f'{score:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# 稳定性分析
print("\n稳定性分析:")
if cv_scores.std() < 0.01:
    print("模型非常稳定，各折之间差异很小")
elif cv_scores.std() < 0.02:
    print("模型稳定，各折之间差异较小")
else:
    print("模型稳定性一般，各折之间有较大差异")

### 6.2 决策边界可视化（简化版）

**任务**: 使用 PCA 降维后可视化决策边界

In [ ]:
# ===== 参考答案 =====
from sklearn.decomposition import PCA

# 使用 PCA 将数据降维到 2D
pca = PCA(n_components=2)
X_train_2d = pca.fit_transform(X_train)
X_test_2d = pca.transform(X_test)

print(f"PCA 解释方差比: {pca.explained_variance_ratio_}")
print(f"累积解释方差: {pca.explained_variance_ratio_.sum():.4f}")

# 在 2D 数据上训练 KNN
knn_2d = KNeighborsClassifier(n_neighbors=best_k)
knn_2d.fit(X_train_2d, y_train)

# 计算边界
x_min, x_max = X_train_2d[:, 0].min() - 1, X_train_2d[:, 0].max() + 1
y_min, y_max = X_train_2d[:, 1].min() - 1, X_train_2d[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

# 预测网格
Z = knn_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# 绘制决策边界
plt.figure(figsize=(14, 6))

# 左图: 训练集
plt.subplot(1, 2, 1)
scatter = plt.scatter(X_train_2d[:, 0], X_train_2d[:, 1], c=y_train,
                       cmap='tab10', alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
plt.contourf(xx, yy, Z, alpha=0.3, cmap='tab10')
plt.xlabel('主成分 1')
plt.ylabel('主成分 2')
plt.title(f'训练集 - PCA 降维后的决策边界 (K={best_k})')
plt.colorbar(scatter, label='数字')

# 右图: 测试集
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_test_2d[:, 0], X_test_2d[:, 1], c=y_test,
                       cmap='tab10', alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
plt.contourf(xx, yy, Z, alpha=0.3, cmap='tab10')
plt.xlabel('主成分 1')
plt.ylabel('主成分 2')
plt.title(f'测试集 - PCA 降维后的决策边界 (K={best_k})')
plt.colorbar(scatter, label='数字')

plt.tight_layout()
plt.show()

# 评估 2D 模型的性能
train_acc_2d = knn_2d.score(X_train_2d, y_train)
test_acc_2d = knn_2d.score(X_test_2d, y_test)
print(f"\n2D PCA 模型性能:")
print(f"训练集准确率: {train_acc_2d:.4f}")
print(f"测试集准确率: {test_acc_2d:.4f}")
print(f"\n注意: PCA 降维后损失了部分信息，准确率会降低")

### 6.3 性能分析

**任务**: 分析 K 值对预测时间的影响

In [ ]:
import time

# 测试不同 K 值的预测时间
k_values = [1, 5, 11, 21, 31, 51]
prediction_times = []

print("测量不同 K 值的预测时间...\n")
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # 测量预测时间
    start = time.time()
    knn.predict(X_test)
    end = time.time()
    
    elapsed = end - start
    prediction_times.append(elapsed)
    print(f"K={k:2d}: 预测时间 = {elapsed:.4f} 秒")

# 绘制结果
plt.figure(figsize=(10, 5))
plt.plot(k_values, prediction_times, 'o-', linewidth=2)
plt.xlabel('K 值')
plt.ylabel('预测时间 (秒)')
plt.title('K 值对预测时间的影响')
plt.grid(True, alpha=0.3)
plt.show()

print("\n结论:")
print("- KNN 的预测时间随着 K 值增加略有增加")
print("- KNN 的主要时间开销是距离计算，这与 K 值关系不大")

## 总结

恭喜你完成了 KNN 实验！在本实验中，你学习了：

1. ✅ **KNN 原理**: 基于最近邻居的多数投票
2. ✅ **距离度量**: 欧氏距离、曼哈顿距离
3. ✅ **手动实现**: 从零实现 KNN 分类器
4. ✅ **scikit-learn KNN**: 使用 KNeighborsClassifier
5. ✅ **K 值选择**: 找到最优的邻居数量
6. ✅ **模型评估**: 准确率、混淆矩阵、交叉验证

### 关键要点

- **K 值选择**: K 太小容易过拟合，K 太大容易欠拟合
- **距离度量**: 欧氏距离最常用，曼哈顿距离对异常值更鲁棒
- **加权投票**: 距离权重通常比均匀权重效果好
- **特征缩放**: KNN 对特征尺度敏感，需要归一化
- **计算成本**: KNN 是惰性学习，预测时需要计算大量距离

### 进一步学习

- 尝试其他距离度量（如余弦相似度）
- 学习 KD 树和 Ball 树等加速算法
- 探索 KNN 在回归问题中的应用 (KNeighborsRegressor)
- 研究维度灾难及其缓解方法